In [1]:
from typing import Any, List

import torch
from pytorch_lightning import LightningModule
from pytorch_lightning.metrics.classification import Accuracy

from lightning_hydra_classifiers.models.modules.simple_dense_net import SimpleDenseNet


import os
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms

import pytorch_lightning as pl


# Replace default file cloud urls from Yann Lecun's website to offiial aws s3 bucket
new_mirror = 'https://ossci-datasets.s3.amazonaws.com/mnist'
MNIST.resources = [
                   ('/'.join([new_mirror, url.split('/')[-1]]), md5)
                   for url, md5 in MNIST.resources
                   ]


if 'TOY_DATA_DIR' not in os.environ: 
    os.environ['TOY_DATA_DIR'] = "/media/data_cifs/projects/prj_fossils/data/toy_data"
        
default_root_dir = os.environ['TOY_DATA_DIR']





class CoolSystem(pl.LightningModule):

    def __init__(self, classes=10):
        super().__init__()
        self.save_hyperparameters()

        self.l1 = torch.nn.Linear(28 * 28, self.hparams.classes)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def prepare_data(self):
        MNIST(default_root_dir, train=True, download=True, transform=transforms.ToTensor())
        MNIST(default_root_dir, train=False, download=True,  transform=transforms.ToTensor())

    def train_dataloader(self):
        mnist_train = MNIST(default_root_dir, train=True, download=False, transform=transforms.ToTensor())
        loader = DataLoader(mnist_train, batch_size=32, num_workers=4)
        return loader

    def test_dataloader(self):
        mnist_test = MNIST(default_root_dir, train=False, download=False, transform=transforms.ToTensor())
        loader = DataLoader(mnist_test, batch_size=32, num_workers=4)
        return loader

    
    
    
    
def main():
    model = CoolSystem()

    # most basic trainer, uses good defaults
    trainer = pl.Trainer(gpus=1, precision=16, progress_bar_refresh_rate=5, max_epochs=10)
    trainer.fit(model)
    
    test_results = trainer.test()
    
    return test_results

/media/data/conda/jrose3/envs/sequoia/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [2]:
%reload_ext tensorboard
%tensorboard --port=9898 --logdir lightning_logs/


main()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 7.9 K 
--------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)


/media/data/conda/jrose3/envs/sequoia/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:99: UserWarning: you passed in a test_dataloader but have no test_step. Skipping test loop
  rank_zero_warn(f'you passed in a {loader_name} but have no {step_name}. Skipping {stage} loop')
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


([], [])